# Pregunta 2. Análisis de opiniones sobre películas.
a) En primer lugar hay que descargar los datos de la URL asignada y luego se guardan en los archivos train_data2.csv y test_data2.csv. 

Luego se abren los archivos y se crea un dataframe para los conjuntos de datos de entrenamiento (train) y prueba (test).

La primera columna de los sets de datos corresponde al sentimiento (Sentiment). Si es "-1", entonces el comentario es negativo. Si es "+1" entonces el comentario será positivo. Esto se pudo comprobar leyendo un extracto del dataframe. 

La segunda columna será el review de la persona con respecto a una película.

In [1]:
import urllib
import pandas as pd

'''
train_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.train"
test_data_url = "http://www.inf.utfsm.cl/~jnancu/stanford-subset/polarity.dev"
train_data_f = urllib.urlretrieve(train_data_url, "train_data2.csv")
test_data_f = urllib.urlretrieve(test_data_url, "test_data2.csv")
'''


ftr = open("train_data2.csv", "r")
fts = open("test_data2.csv", "r")
rows = [line.split(" ",1) for line in ftr.readlines()]
train_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
train_df['Sentiment'] = pd.to_numeric(train_df['Sentiment'])
rows = [line.split(" ",1) for line in fts.readlines()]
test_df = pd.DataFrame(rows, columns=['Sentiment','Text'])
test_df['Sentiment'] = pd.to_numeric(test_df['Sentiment'])
print train_df.shape
print test_df.shape

(3554, 2)
(3554, 2)


Finalmente, se tiene que tanto el conjunto de entrenamiento como el de pruebas tienen igual cantidad de registros (3554 filas).

b) Ahora hay que crear una función que entregue una lista de palabras a partir de un determinado texto. A continuación se muestra el código del enunciado, en el cual realiza lematización. Notar que la función ya incluye su debida transformación a lower-case.

In [21]:
import re, time
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer, word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.lancaster import LancasterStemmer
def word_extractor_lemmatizer(text):
    wordlemmatizer = WordNetLemmatizer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ wordlemmatizer.lemmatize(word.lower()) for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words

print word_extractor_lemmatizer("I love to eat cake")
print word_extractor_lemmatizer("I love eating cake")
print word_extractor_lemmatizer("I loved eating the cake")
print word_extractor_lemmatizer("I do not love eating cake")
print word_extractor_lemmatizer("I don't love eating cake")

 love eat cake
 love eating cake
 loved eating cake
 love eating cake
 n't love eating cake


El siguiente código es el modificado que incluye el proceso de stemming.

In [27]:
def word_extractor(text):
    stemmer = PorterStemmer()
    commonwords = stopwords.words('english')
    text = re.sub(r'([a-z])\1+', r'\1\1',text)#substitute multiple letter by two
    words = ""
    wordtokens = [ stemmer.stem(word.lower()) for word in word_tokenize(text.decode('utf-8', 'ignore')) ]
    for word in wordtokens:
        if word not in commonwords:
            words+=" "+word
    return words

print word_extractor("I love to eat cake")
print word_extractor("I love eating cake")
print word_extractor("I loved eating the cake")
print word_extractor("I do not love eating cake")
print word_extractor("I don't love eating cake")

 love eat cake
 love eat cake
 love eat cake
 love eat cake
 n't love eat cake


Se observa una gran diferencia ya que, para la función que incluye lematización, se puede ver a simple vista que solo se eliminaron las stop-words. En cambio, para la función con stemming, las primeras cuatro frases quedan como "love eat cake", ya que las palabras "loved" y "eating" se redujeron a su raiz "love" y "eat" respectivamente.

Probando con otras frases...

In [28]:
print word_extractor("I'm playing PlayStation 4 every week")
print word_extractor("I play PlayStation 4 every week")

print word_extractor("I learned a lot in the Machine Learning class")
print word_extractor("I learn a lot from the Machine Learning classes")


 'm play playstat 4 everi week
 play playstat 4 everi week
 learn lot machin learn class
 learn lot machin learn class


En los ejemplos anteriores, se ve que para las dos primeras frases la función trata al nombre propio "PlayStation" como una palabra más, y le borra el sufijo "ion", quedando "playstat". 

Para las otras dos frases, cada una entrega un mensaje distinto. La primera comunica que el sujeto aprendió harto de la clase de Machine Learning. En la segunda frase, el sujeto informa que aprende harto de las clases de Machine Learning. Aún así, luego del proceso de stemming ambas frases quedan iguales.

c) La función word_extractor_lemmatizer() trata las palabras por lematización y fue realizada anteriormente. A continuación se muestran los últimos ejemplos anteriores utilizando lematización.

In [30]:
print word_extractor_lemmatizer("I'm playing PlayStation 4 every week")
print word_extractor_lemmatizer("I play PlayStation 4 every week")

print word_extractor_lemmatizer("I learned a lot in the Machine Learning class")
print word_extractor_lemmatizer("I learn a lot from the Machine Learning classes")

print word_extractor_lemmatizer("better")

 'm playing playstation 4 every week
 play playstation 4 every week
 learned lot machine learning class
 learn lot machine learning class
 better


Técnicamente, el proceso de stemming lo que hace es cortar el final de las palabras sin importar el significado de la palabra según el contexto. Lemmatization (o lematización) es un proceso mas complejo, ya que utiliza una búsqueda en un vocabulario y realiza un análisis morfológico de las palabras.

In [40]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

'''def vector(tipo='lemmatizer'):
  if (tipo == 'lemmatizer'):'''
texts_train = [word_extractor_lemmatizer(text) for text in train_df.Text]
texts_test = [word_extractor_lemmatizer(text) for text in test_df.Text]
'''elif tipo == 'stemming':
    texts_train = [word_extractor(text) for text in train_df.Text]
    texts_test = [word_extractor(text) for text in test_df.Text]'''

vectorizer = CountVectorizer(ngram_range=(1, 1), binary='False')
vectorizer.fit(np.asarray(texts_train))

features_train = vectorizer.transform(texts_train)
features_test = vectorizer.transform(texts_test)

labels_train = np.asarray((train_df.Sentiment.astype(float)+1)/2.0)
labels_test = np.asarray((test_df.Sentiment.astype(float)+1)/2.0)

vocab = vectorizer.get_feature_names()
dist=list(np.array(features_train.sum(axis=0)).reshape(-1,))
for tag, count in zip(vocab, dist):
    print count, tag

IndentationError: expected an indented block (<ipython-input-40-e297d9877bb1>, line 24)

In [37]:
vector()